In [1]:
import cv2
import torch
from torchvision.transforms import transforms

from Enviroment import Enviroment
import cv2
import numpy as np

/home/tatiana/anaconda3/envs/py9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pygame 2.1.3.dev8 (SDL 2.0.22, Python 3.9.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
env = Enviroment(obstacle=True, Viz=True, War=True,
                head_velocity=0.0, num_obs=5, num_enemy=1, 
                size_obs=[50, 60], m_step=500,
                in_collision_rew=-70, in_win_rew=100, in_defeat_rew=-100,
                optionEnemy=[45,90], optionAlie=[40, 90],
                epsilon = 100, sigma = 30)

In [3]:
class Planer():
    def __init__(self, path, env):

        self.model = torch.load(path, map_location='cuda')
        self.env = env
        self.transforms = transforms.Compose([transforms.ToTensor()]) #,
                 # transforms.Normalize(mean = (0.5), std = (0.5))])
        self.model.eval()
        
        
        
    def get_statistics(self, num_games):
        colision = 0
        win = 0
        lose = 0
        loop = 0
        
        for game in range(num_games):
            
            done = False
            state = self.env.reset()
            img1 = state.img
            img2 = state.img
            img3 = state.img
            Img_final = None
            while not done:
                
                img1 = img2
                img2 = img3
                img3 = state.img
                new_img = cv2.addWeighted(img2, 0.4, img1, 0.2, 0)
                Img_final = cv2.addWeighted(img3, 0.7, new_img, 0.5, 0)

                # Img_final = cv2.resize(Img_final, (65,65), interpolation = cv2.INTER_AREA)
                
                # cv2.imwrite('img.png', Img_final)
                # img0 = cv2.imread('img.png')

                img = self.transforms(Img_final).unsqueeze(0).to('cuda')
                model_out = self.model(img)
                act = int(torch.argmax(torch.softmax(model_out, dim=1)))

                state, rew, done, _ = self.env.step(act)
            
            if rew[0] == self.env.rew_collision:
                colision += 1
            elif rew[0] == self.env.rew_defeat:
                lose += 1
            elif rew[0] == self.env.rew_win:
                win += 1
            else:
                loop += 1
                
        print(num_games, 'games were played.')
        print('win', win/num_games)
        print('colision', colision/num_games)
        print('loop', loop/num_games)
        print('lose', lose/num_games)
        
    def render(self, num_games):
        for i in range(num_games):
            
            images = []
            obs = self.env.reset()
            
            img1 = obs.img
            img2 = obs.img
            img3 = obs.img
            Img_final = None
            img = obs.img # env.render(mode='rgb_array')
            done = False
                
            height, width, layers = img.shape
            size = (width,height)
            out = cv2.VideoWriter(f"video{i}.avi",cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            out.write(img)
            while not done:
                
                img1 = img2
                img2 = img3
                img3 = obs.img
                new_img = cv2.addWeighted(img2, 0.4, img1, 0.2, 0)
                Img_final = cv2.addWeighted(img3, 0.7, new_img, 0.5, 0)
                
                # img = cv2.resize(Img_final, (65,65), interpolation = cv2.INTER_AREA)

                # cv2.imwrite('img.png', Img_final)
                # img0 = cv2.imread('img.png')
                
                img = self.transforms(Img_final).unsqueeze(0).to('cuda')
                
                model_out = self.model(img.to('cuda'))
                action = int(torch.argmax(torch.softmax(model_out, dim=1)))
                obs, _, done ,_ = self.env.step(action)
                img = obs.img
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                out.write(img)
            out.release()

In [4]:
plan = Planer('./res18_lr29',  env=env)

In [5]:
plan.get_statistics(1000)

1000 games were played.
win 0.23
colision 0.667
loop 0.024
lose 0.079


In [11]:
# plan.render(20)